In [1]:
import math, re, os
import numpy as np
import tensorflow as tf
from matplotlib import pyplot as plt   # matplotlib进行画图
from kaggle_datasets import KaggleDatasets # Kaggle数据集
import efficientnet.tfkeras as efn    # 导入efficientnet模型
# 从python的sklearn机器学习中导入f1值、精度、召回率和混淆矩阵
from sklearn.metrics import f1_score, precision_score, recall_score, confusion_matrix 
print("Tensorflow version " + tf.__version__)

Tensorflow version 2.5.0


In [ ]:
AUTO = tf.data.experimental.AUTOTUNE # 可以让程序自动的选择最优的线程并行个数
# 从TPU创建部署
tpu = tf.distribute.cluster_resolver.TPUClusterResolver() #如果先前设置好了ＴＰＵ＿ＮＡＭＥ环境变量，不需要再给参数．
tf.config.experimental_connect_to_cluster(tpu) # 配置实验连接到群集
tf.tpu.experimental.initialize_tpu_system(tpu) # 初始化tpu系统
strategy = tf.distribute.experimental.TPUStrategy(tpu) # 设置TPU部署
GCS_DS_PATH = KaggleDatasets().get_gcs_path('tpu-getting-started')
